## Análise pós-clusterização
***

In [469]:
import pandas as pd
import numpy as np

In [475]:
pd.read_csv(path + 'var_quanti_agg_pre.csv', sep = ',').head()

,municipio,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,freq,share_tracado_via_tipo_1,share_tracado_via_tipo_2,share_is_night,share_is_weekend,share_is_single_lane
0,curitiba,53273,581,10382,2872,36629,2808,13254,46614,23714,0.834781,0.155604,0.375221,0.438306,0.106477
1,sao jose,44027,191,8828,1928,31326,1752,10756,40653,20102,0.784648,0.205552,0.369018,0.414785,0.043379
2,guarulhos,40350,430,8282,1040,27409,3188,9322,36372,18109,0.942239,0.052018,0.410293,0.422055,0.055221
3,betim,39527,329,6840,1718,29551,1085,8558,35159,17601,0.784103,0.208625,0.361911,0.489234,0.048804
4,duque de caxias,37859,407,6388,1273,28066,1723,7661,32465,17531,0.842393,0.153785,0.380811,0.430152,0.195710


In [471]:
path = '/Users/felipebarreto/Desktop/TCC/Bases geradas/'
df1 = pd.read_csv('/Users/felipebarreto/Desktop/dados_acidentes_tratados_completo.csv', sep = ',')   ## dados originais tratados
df2 = pd.read_csv(path + 'centroides_padronizados.csv', sep = ';')   ## centróides de cada cluster     //
df3 = pd.read_csv(path + 'variaveis_quanti_agg.csv', sep = ';')      ## igual ao df4, sem padronização // 
df4 = pd.read_csv(path + 'variaveis_quanti_pad_agg.csv', sep = ';')  ## igual ao df3, com padronização // 
df5 = pd.read_csv(path + 'medias_var_cluster.csv', sep = ';', decimal= '.')        ## médias de cada cluster         // 
df6 = pd.read_csv(path + 'clusters_7.csv', sep = ';') # var quantitativas completas, nem todas foram utilizadas para a clusterização

In [457]:
def _status_qtd(var):
      """
      Função para cálculo de frequência e porcentagem relativa a cada grupo

      Parâmetro:
      ----------
      var - variável a ser sumarizada e normalizada
      """
      a = pd.merge(df1, df6[['municipio','cluster']], how='left')\
            .groupby(['cluster',var])[['id']]\
            .count()\
            .reset_index()\
            .rename(columns={var:'status','id':var})\
            .set_index(['cluster','status'])
      a_pct = a.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))\
            .rename(columns={var:var+'_pct'})
      df_completo = a.join(a_pct)
      return df_completo

In [458]:
## Crisndo um dataframe com o share das variáveis categóricas
df_share = _status_qtd('tracado_via_tipo_1')\
    .join(_status_qtd('is_night'))\
    .join(_status_qtd('is_weekend'))\
    .join(_status_qtd('is_single_lane'))\
    .reset_index()\
    .query("status == True")

In [445]:
display(
    df2.style.background_gradient(cmap='Greens').set_precision(2),
    df5.style.background_gradient(cmap='Reds').set_precision(2),
    df_share.style.background_gradient(cmap='Blues').set_precision(2)
    )

,Variavel,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5
0,mortos,3.51,1.04,-0.69,-0.34,-0.27
1,feridos_leves,4.22,0.84,-0.48,-0.28,-0.30
2,feridos_graves,4.05,0.87,-0.52,-0.30,-0.29
3,ilesos,4.55,0.64,-0.39,-0.27,-0.28
4,freq,4.54,0.74,-0.43,-0.28,-0.30
5,share_tracado_via_tipo_1,0.20,0.04,0.33,-1.28,0.51
6,share_is_night,-0.22,-0.09,-1.76,-0.02,0.25
7,share_is_weekend,-0.18,-0.02,-2.24,0.16,0.20
8,share_is_single_lane,-1.61,-0.95,-0.18,0.22,0.29


,cluster,mortos,feridos_leves,feridos_graves,ilesos,freq,share_tracado_via_tipo_1,share_is_night,share_is_weekend,share_is_single_lane
0,Cluster 1,239.19,3635.01,1149.28,13374.51,8837.81,0.74,0.37,0.45,0.31
1,Cluster 2,98.84,1019.67,349.91,2796.47,2074.71,0.70,0.39,0.48,0.52
2,Cluster 3,0.37,1.47,0.65,4.02,3.08,0.78,0.07,0.06,0.76
3,Cluster 4,20.29,155.51,56.54,326.65,278.06,0.39,0.41,0.51,0.89
4,Cluster 5,24.20,135.33,58.91,305.96,239.30,0.82,0.46,0.52,0.91


,cluster,status,tracado_via_tipo_1,tracado_via_tipo_1_pct,is_night,is_night_pct,is_weekend,is_weekend_pct,is_single_lane,is_single_lane_pct
1,1,True,446444,75.40,216439,36.55,263260,44.46,155120,26.20
3,2,True,496170,69.52,276946,38.80,339562,47.58,364725,51.10
5,3,True,309,74.82,53,12.83,75,18.16,343,83.05
7,4,True,64097,43.58,55034,37.41,71434,48.56,131584,89.46
9,5,True,221883,77.98,120169,42.23,139301,48.96,257011,90.33


### Análise

In [446]:
df = df3.drop(['pessoas','ignorados','feridos','veiculos'], axis=1)[[
    'municipio','cluster','freq','mortos','feridos_graves','feridos_leves','ilesos','share_tracado_via_tipo_1',
    'share_tracado_via_tipo_2','share_is_night','share_is_weekend','share_is_single_lane'
]].drop_duplicates(subset = ['municipio'])


## Separação dos clusters (c1,c2,c3,c4,c5,c6,c7)
n = df.cluster.nunique()
for i in range(1,n + 1):
    globals()['c' + str(i)] = df[df.cluster == i].drop(['cluster'], axis=1)

df.head(5)

,municipio,cluster,freq,mortos,feridos_graves,feridos_leves,ilesos,share_tracado_via_tipo_1,share_tracado_via_tipo_2,share_is_night,share_is_weekend,share_is_single_lane
0,curitiba,1,23714,581,2872,10382,36629,0.834781,0.155604,0.375221,0.438306,0.106477
1,sao jose,1,20102,191,1928,8828,31326,0.784648,0.205552,0.369018,0.414785,0.043379
2,guarulhos,1,18109,430,1040,8282,27409,0.942239,0.052018,0.410293,0.422055,0.055221
3,betim,1,17601,329,1718,6840,29551,0.784103,0.208625,0.361911,0.489234,0.048804
4,duque de caxias,1,17531,407,1273,6388,28066,0.842393,0.153785,0.380811,0.430152,0.195710


In [447]:
def _loc(df, cluster, col):
    """
    Função para buscar um valor específico na tabela, dada uma variável

    Parâmetros:
    -----------
    df ------> dataframe 
    cluster -> cluster escolhido
    col -----> variável a ser buscada
    """
    return(round(df[df.cluster == 'Cluster '+str(cluster)][col].iloc[0],2))

In [451]:
## Tamanho dos clusters // Quantidade de municípios
print(
    f""">> DADOS GERAIS - Acidentes de 2007 a 2021 (16 anos)<<
    - Total de municípios analisados: {len(df)}
    - Mortos = {df.mortos.sum()} | Média = {round(df.mortos.mean(),2)}
    - Feridos graves = {df.feridos_graves.sum()} | Média = {round(df.feridos_graves.mean(),2)}
    - Feridos leves = {df.feridos_leves.sum()} | Média = {round(df.feridos_leves.mean(),2)}
    - Ilesos = {df.ilesos.sum()} | Média = {round(df.ilesos.mean(),2)}
    - Share de tração via tipo 1 = {round(100*df1.tracado_via_tipo_1.value_counts(normalize=True)[1],2)}%
    - Share de é noite = {round(100*df1.is_night.value_counts(normalize=True)[1],2)}%
    - Share de é final de semana = {round(100*df1.is_weekend.value_counts(normalize=True)[1],2)}%
    - Share de é pista simples = {round(100*df1.is_single_lane.value_counts(normalize=True)[1],2)}%
    """
)

for i in range(1,n+1):

    print(f"""Cluster {i}:
        - {len(globals()['c' + str(i)])} municípios --> {round(100*df[df.cluster == i].shape[0] / df.shape[0],2)}% do total
        - Mortos = {globals()['c' + str(i)].mortos.sum()},  {round(100*globals()['c' + str(i)].mortos.sum() / df.mortos.sum(),2)}% do total | Média = {_loc(df5, i, col='mortos')}, {round(100*_loc(df5, i, col='mortos') / df.mortos.mean(),2)}% da média geral
        - Feridos graves = {globals()['c' + str(i)].feridos_graves.sum()},  {round(100*globals()['c' + str(i)].feridos_graves.sum() / df.feridos_graves.sum(),2)}% do total | Média = {_loc(df5, i, col='feridos_graves')}, {round(100*_loc(df5, i, col='feridos_graves') / df.feridos_graves.mean(),2)}% da média geral
        - Feridos Leves = {globals()['c' + str(i)].feridos_leves.sum()},  {round(100*globals()['c' + str(i)].feridos_leves.sum() / df.feridos_leves.sum(),2)}% do total | Média = {_loc(df5, i, col='feridos_leves')}, {round(100*_loc(df5, i, col='feridos_leves') / df.feridos_leves.mean(),2)}% da média geral
        - Ilesos = {globals()['c' + str(i)].ilesos.sum()},  {round(100*globals()['c' + str(i)].ilesos.sum() / df.ilesos.sum(),2)}% do total | Média = {_loc(df5, i, col='ilesos')}, {round(100*_loc(df5, i, col='ilesos') / df.ilesos.mean(),2)}% da média geral
        - Média do share de traçado via tipo 1 = {round(100*_loc(df5, i, col='share_tracado_via_tipo_1'),2)}%
        - Média do share de é noite = {round(100*_loc(df5, i, col='share_is_night'),2)}%
        - Média do share de é final de semana = {round(100*_loc(df5, i, col='share_is_weekend'),2)}%
        - Média do share de é pista simples = {round(100*_loc(df5, i, col='share_is_single_lane'),2)}%
    """)

>> DADOS GERAIS - Acidentes de 2007 a 2021 (16 anos)<<
    - Total de municípios analisados: 2263
    - Mortos = 89587 | Média = 39.59
    - Feridos graves = 297414 | Média = 131.42
    - Feridos leves = 837677 | Média = 370.16
    - Ilesos = 2395201 | Média = 1058.42
    - Share de tração via tipo 1 = 70.71%
    - Share de é noite = 38.47%
    - Share de é final de semana = 46.82%
    - Share de é pista simples = 52.29%
    
Cluster 1:
        - 67 municípios --> 2.96% do total
        - Mortos = 16026,  17.89% do total | Média = 239.19, 604.2% da média geral
        - Feridos graves = 77002,  25.89% do total | Média = 1149.28, 874.48% da média geral
        - Feridos Leves = 243546,  29.07% do total | Média = 3635.01, 982.0% da média geral
        - Ilesos = 896092,  37.41% do total | Média = 13374.51, 1263.63% da média geral
        - Média do share de traçado via tipo 1 = 74.0%
        - Média do share de é noite = 37.0%
        - Média do share de é final de semana = 45.0%
       

#### Observações:

- Total de Municípios analisados: 2263
- Municípios por região:
    - N ---> 
    - NE --> 
    - CO --> 
    - S ---> 
    - SE -->  
- Municípios por UF:
    - sdc

## Cluster 1

In [453]:
for i in range(1,6):
    print(f"""
        Freq Cluster {i}: {globals()['c'+str(i)].freq.sum()}
    """)


        Freq Cluster 1: 592133
    

        Freq Cluster 2: 713701
    

        Freq Cluster 3: 413
    

        Freq Cluster 4: 147093
    

        Freq Cluster 5: 284529
    


In [455]:
df1.mortos.sum()

89587